In [13]:
programming_languages = [
    "Python 2",
    "Python 3", # Moderne Variante von Python (Python 2 > wird kaum noch genutzt)
    "Java",
    "JavaScript", # Auch in modernen Varianten (ES6+ etc.)
    "TypeScript", # „Superset“ von JavaScript
    "C",
    "C++", # In verschiedenen Varianten (C++98, C++11, C++14, C++17, C++20 – hier pauschal)
    "C#", # .NET-Sprache
    "PHP", # Auch wenn neuere Versionen (7, 8) erheblich verbessert wurden
    "Ruby",
    "Swift",
    "Kotlin",
    "Objective-C", # Vor allem im Apple-Umfeld (neben Swift)
    "Go", # Google’s Sprache für verteilte Systeme
    "Rust", # Moderne, sicherheitsorientierte Systemsprache
    "R", # Wichtig in Statistik und Data Science
    "MATLAB", # Stark im wissenschaftlichen Bereich und Engineering
    "Perl", # Historisch relevant; in einigen Legacy-Projekten und nichespezifischen Anwendungen immer noch genutzt
    "Scala", # Hybrid aus objektorientierter und funktionaler Programmierung (z. B. in Big-Data-Umgebungen)
    "Haskell", # Stark typisierte funktionale Programmiersprache
    "Lua", # Eingebettete Skriptsprache, z. B. in Games
    "Clojure", # Moderne Lisp-Variante, läuft auf der JVM
    "Elixir", # Moderne Sprache für verteilte und fehlertolerante Systeme, basiert auf Erlang VM
    "Dart", # Vor allem für Flutter (mobile und Web-Apps)
    "Visual Basic .NET", # Anwendungsentwicklung im Microsoft-Umfeld
    "SQL", # Domänenspezifische Sprache für Datenbankabfragen
    "Bash", # Standard-Shell-Skriptsprache in Unix/Linux-Umgebungen
    "PowerShell", # Skriptsprache und Shell für Windows, mittlerweile auch plattformübergreifend
    "COBOL", # Veraltete Syntax, aber in vielen Finanz- und Verwaltungssystemen nach wie vor im Einsatz
    "Fortran", # Besonders in numerischer Berechnung und wissenschaftlichen Anwendungen weiterhin wichtig
    "Assembly", # Direkte Maschinenprogrammierung – offensichtlich nur für spezielle Einsatzzwecke
    "Groovy", # Alternative JVM-Sprache, häufig in Build-Skripten (z. B. Gradle) verwendet
    "F#", # Funktionale Sprache im .NET-Umfeld
    "Erlang", # Zuverlässige, fehlerresistente Systeme (vor allem Telekommunikation)
    "Julia", # Moderne Sprache für numerische und wissenschaftliche Berechnungen
    "D", # Systemnahe Sprache, weniger verbreitet als C++/Rust, aber noch in Nischen im Einsatz
    "ABAP", # Hauptsächlich in SAP-Umgebungen
    "Ada", # Robustheit und Sicherheit – vor allem in sicherheitskritischen Bereichen (z. B. Luftfahrt & Verteidigung)
    "Prolog" # Logikprogrammierung, mehr in Nischen und Forschung, aber trotzdem im Einsatz
    ]


In [15]:
snippets = {}

In [16]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv("../.env")
client = OpenAI()

if not len(snippets):
    for language in programming_languages:
        prompt = f"Write a not too common sample snippet in {language}. Provide only the snippet and don't explain yourself. It should be a very short example. Here comes just your answer: "

        # Generate completions for each programming language
        completions = client.chat.completions.create(
            model="o3-mini-2025-01-31",
            messages=[    
                {"role": "developer", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
                ]
        )
        snippet = completions.choices[0].message.content
        # Print the completions
        print(f"Language: {language}\nSnippet:\n{snippet}\n\n\n")
        snippets[language] = snippet

Language: Python 2
Snippet:
ChatCompletionMessage(content='def compare_length(s1, s2):\n    return cmp(len(s1), len(s2))\n\nwords = ["apple", "banana", "kiwi", "grape"]\nprint sorted(words, cmp=compare_length)', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)



Language: Python 3
Snippet:
ChatCompletionMessage(content='def flatten(lst):\n    for item in lst:\n        if isinstance(item, list):\n            yield from flatten(item)\n        else:\n            yield item\n\ndata = [1, [2, [3, 4], 5], 6]\nprint(list(flatten(data)))', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)



Language: Java
Snippet:
ChatCompletionMessage(content='public class RecursiveFactorial {\n    public static void main(String[] args) {\n        java.util.function.Function<Integer, Integer> factorial = new java.util.function.Function<Integer, Integer>() {\n            @Override\n            public Integer apply(Integer n) {\n                re

APIConnectionError: Connection error.

In [ ]:
for lang, snippet in snippets.items():
    snippets[lang] = snippet.content

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = "cpu"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

messages = [{"role": "user", "content": "What is gravity?"}]

input_text=tokenizer.apply_chat_template(messages, tokenize=False)
print(input_text)

inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=50, temperature=0.2, top_p=0.9, do_sample=True)

print(tokenizer.decode(outputs[0]))


for lang, snippet in snippets:
    messages = [{"role": "user", "content": f"Q: Given the source code below:\n```{snippet}```\n\n Which programming language is this? You must only name the programming language and no explanations. Permitted answer formats are 'Python 2', 'Python 3', 'Java', 'C#'.\n\n A: "}]

    input_text=tokenizer.apply_chat_template(messages, tokenize=False)

    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=50, temperature=0.2, top_p=0.9, do_sample=True)

    
    print(tokenizer.decode(outputs[0]).split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0], " ?= ", lang, "  - #tokens =", inputs.shape[1])
    